In [1]:
from bs4 import BeautifulSoup
import re

In [ ]:
! curl -o data/htlms/'15'_september.html "https://www.cdep.ro/pls/steno/steno2015.stenograma?ids=8928&idl=1"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  163k  100  163k    0     0   179k      0 --:--:-- --:--:-- --:--:--     0--:-- --:--:--  179k


In [ ]:
html_file = 'data/htmls/15_september.html'
try:
    with open(html_file, 'r', encoding='iso-8859-2') as f:
        html_content = f.read()
    soup = BeautifulSoup(html_content, 'html.parser')

except FileNotFoundError:
    print(f"Error: The file '{html_file}' was not found.")
except UnicodeDecodeError as e:
    print(f"UnicodeDecodeError: {e}")
    print(f"There was an issue decoding the file. Ensure it is saved with 'iso-8859-2' encoding or try other common encodings if 'iso-8859-2' also fails (e.g., 'latin1', 'windows-1250').")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [22]:
def extract_meeting_data(soup):
    data = {
        'title': None,
        'intro': None,
        'conversation': ""
    }

    # extract title
    title_h3 = soup.select_one('div.content-right div.box-title.clearfix h3')
    if title_h3:
        data['title'] = title_h3.get_text(strip=True)

    old_div = soup.find('div', id='olddiv')
    if not old_div:
        return data
        
    transcript_table_body = old_div.select_one('table > tbody')
    if not transcript_table_body:
        transcript_table_body = old_div.find('table')
        if not transcript_table_body:
            print("Error: Could not find the main transcript table or its tbody.")
            return data

    rows = transcript_table_body.find_all('tr', valign='top', recursive=False)
    if not rows:
        return data

    # extract intro
    intro_texts_collected = []
    intro_section_identified = False
    if rows:
        intro_row_td = rows[0].find('td', {'width': '100%'})
        if intro_row_td:
            intro_paragraphs = intro_row_td.find_all('p', align='justify')
            
            temp_intro_check_text = ' '.join([p.get_text(strip=True) for p in intro_paragraphs])
            if "Şedinţa a început" in temp_intro_check_text or "Lucrările şedinţei au fost conduse" in temp_intro_check_text:
                intro_section_identified = True
            
            if intro_section_identified:
                for p in intro_paragraphs:
                    intro_texts_collected.append(p.get_text(separator=' ', strip=True))
                if intro_texts_collected:
                    data['intro'] = ' '.join(intro_texts_collected)

    # extract conversation
    all_turn_text_blocks = []
    conversation_rows_start_index = 0
    if data['intro'] and intro_section_identified: 
        conversation_rows_start_index = 1

    for i in range(conversation_rows_start_index, len(rows)):
        row = rows[i]
        # print(row)
        # break
        content_td = row.find('td', {'width': '100%'})
        if not content_td:
            continue

        # Get text from all <p align="justify"> tags within this specific 'td'
        # This will include speaker names and their dialogue as it appears in the HTML paragraphs
        paragraphs_in_td = content_td.find_all('p', align='justify')

        if paragraphs_in_td:
            text = paragraphs_in_td[0].get_text(separator=' ', strip=True)
            if text:
                all_turn_text_blocks.append(text)

    if all_turn_text_blocks:
        # Join the text blocks from each turn/TD with a double newline for separation
        data['conversation'] = "\n\n".join(all_turn_text_blocks)
    
    return data

In [23]:
data = extract_meeting_data(soup)

In [24]:
data['title']

'Sedinta Camerei Deputatilor din 15 septembrie 2025'

In [25]:
data['intro']

'Şedinţa a început la ora 16.06. Lucrările şedinţei au fost conduse, în prima parte, de doamna deputat Natalia-Elena Intotero, vicepreşedinte al Camerei Deputaţilor, asistată de doamna deputat Patricia-Simina-Arina Moş şi de domnul deputat Ovidiu Victor Ganţ, secretari ai Camerei Deputaţilor. A doua parte a şedinţei a fost condusă de domnul deputat Cătălin Drulă, vicepreşedinte al Camerei Deputaţilor. Lucrările şedinţei au fost conduse, în prima parte, de doamna deputat Natalia-Elena Intotero, vicepreşedinte al Camerei Deputaţilor, asistată de doamna deputat Patricia-Simina-Arina Moş şi de domnul deputat Ovidiu Victor Ganţ, secretari ai Camerei Deputaţilor. A doua parte a şedinţei a fost condusă de domnul deputat Cătălin Drulă, vicepreşedinte al Camerei Deputaţilor. A doua parte a şedinţei a fost condusă de domnul deputat Cătălin Drulă, vicepreşedinte al Camerei Deputaţilor.'

In [26]:
data['conversation']

'Doamna Natalia-Elena Intotero : Doamnelor şi domnilor deputaţi, Declar deschisă şedinţa de astăzi a Camerei Deputaţilor şi anunţ că, din totalul celor 330 deputaţi, până în acest moment, şi-au înregistrat prezenţa un număr de 190. Şedinţa se desfăşoară prin mijloace electronice, în format mixt de prezenţă, fizic şi online. În conformitate cu prevederile art. 94 din Regulament, vă informez că au fost distribuite electronic şi afişate pe pagina de Internet a Camerei Deputaţilor următoarele documente: - ordinea de zi pentru şedinţele Camerei Deputaţilor din zilele de luni, 15 şi miercuri, 17 septembrie 2025; - programul de lucru pentru perioada 15 - 20 septembrie 2025; - lista rapoartelor depuse în perioada 8 - 15 septembrie 2025 de comisiile permanente sesizate în fond; - lista cu legile pentru care se poate exercita dreptul de sesizare a Curţii Constituţionale; - informarea cu privire la iniţiativele legislative înregistrate la Camera Deputaţilor şi care urmează să fie avizate de comis

In [27]:
pattern = r'(Domnul|Doamna)\s([^:]+?)\s?:\s(.*?)(?=\n(Domnul|Doamna)\s[^:]+?\s?:|\Z)'
matches = re.findall(pattern, data['conversation'], re.DOTALL)

speeches = []
for match in matches:
    full_name = f"{match[0]} {match[1].strip()}"
    speech_content = match[2].strip()
    speeches.append({"speaker": full_name, "speech": speech_content})

In [28]:
speeches

[{'speaker': 'Doamna Natalia-Elena Intotero',
  'speech': 'Doamnelor şi domnilor deputaţi, Declar deschisă şedinţa de astăzi a Camerei Deputaţilor şi anunţ că, din totalul celor 330 deputaţi, până în acest moment, şi-au înregistrat prezenţa un număr de 190. Şedinţa se desfăşoară prin mijloace electronice, în format mixt de prezenţă, fizic şi online. În conformitate cu prevederile art. 94 din Regulament, vă informez că au fost distribuite electronic şi afişate pe pagina de Internet a Camerei Deputaţilor următoarele documente: - ordinea de zi pentru şedinţele Camerei Deputaţilor din zilele de luni, 15 şi miercuri, 17 septembrie 2025; - programul de lucru pentru perioada 15 - 20 septembrie 2025; - lista rapoartelor depuse în perioada 8 - 15 septembrie 2025 de comisiile permanente sesizate în fond; - lista cu legile pentru care se poate exercita dreptul de sesizare a Curţii Constituţionale; - informarea cu privire la iniţiativele legislative înregistrate la Camera Deputaţilor şi care urmea